# Strands를 사용한 단백질 설계 에이전트

이 노트북에서는 오픈 소스 Strands Agents 프레임워크를 사용하여 단백질 설계 에이전트를 만들어보겠습니다.

#### Strands 에이전트 및 필수 종속성 설치

In [ ]:
%pip install strands-agents strands-agents-tools --quiet

#### 아래에 표시된 최신 버전의 boto3 확인
아래에 출력된 boto3 버전이 **1.37.1** 이상인지 확인하세요.

In [ ]:
%pip show boto3

#### 필수 라이브러리 가져오기

In [ ]:
import boto3
import json
import time
import uuid
import os
from typing import Dict, Any, Optional
from strands import Agent, tool
from strands.models import BedrockModel
from urllib.parse import urlparse

# AWS 계정 정보 가져오기
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']



In [ ]:
session = boto3.Session(
    #aws_access_key_id='your_access_key',
    #aws_secret_access_key='your_secret_key',
    #aws_session_token='your_session_token',  # If using temporary credentials
    #region_name='us-west-2',
    #profile_name='your-profile'  # Optional: Use a specific profile
)
region = session.region_name

## 전제 조건

[00-setup_environment.ipynb](00-setup_environment.ipynb)에서 노트북 환경 설정을 실행하세요.

또한 `stacks/protein_design_stack.yaml`에서 단백질 설계 CloudFormation 스택을 배포했는지 확인하세요.

#### AWS 클라이언트 및 구성 설정
도구에서 사용할 AWS 서비스에 대한 클라이언트를 정의합니다.

In [ ]:
# AWS 클라이언트 초기화
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
omics_client = boto3.client('omics')
s3_client = boto3.client('s3')

# 구성 - CloudFormation 스택 출력을 기반으로 이 값들을 업데이트하세요
STACK_NAME = '{stack_name}'  # 여러분의 CloudFormation 스택 이름
DEFAULT_WORKFLOW_ID = None  # 스택 출력에서 검색됩니다
DEFAULT_ROLE_ARN = None     # 스택 출력에서 검색됩니다
DEFAULT_S3_BUCKET = None    # 스택 출력에서 검색됩니다

# 도구에서 사용할 환경 변수 설정
os.environ['STACK_NAME'] = STACK_NAME

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Stack Name: {STACK_NAME}")

#### CloudFormation 스택 출력 가져오기

In [ ]:
# CloudFormation 스택 출력 가져오기
cf_client = boto3.client('cloudformation')

try:
    response = cf_client.describe_stacks(StackName=STACK_NAME)
    stack = response['Stacks'][0]

    # 출력에서 가져오기
    outputs = stack.get('Outputs', [])
    for output in outputs:
        key = output['OutputKey']
        value = output['OutputValue']
        if key == 'WorkflowId':
            DEFAULT_WORKFLOW_ID = value
        elif key == 'WorkflowExecutionRoleArn':
            DEFAULT_ROLE_ARN = value
            
    parameters = stack.get('Parameters', [])
    for param in parameters:
        if param['ParameterKey'] == 'S3BucketName':
            DEFAULT_S3_BUCKET = param['ParameterValue']
            break
    
    print(f"Workflow ID: {DEFAULT_WORKFLOW_ID}")
    print(f"Role ARN: {DEFAULT_ROLE_ARN}")
    print(f"S3 Bucket: {DEFAULT_S3_BUCKET}")
    
    # 스택 정보로 도구 구성
    from utils.protein_design_tools import set_stack_config
    set_stack_config(
        stack_name=STACK_NAME,
        workflow_id=DEFAULT_WORKFLOW_ID,
        role_arn=DEFAULT_ROLE_ARN,
        s3_bucket=DEFAULT_S3_BUCKET
    )
    
except Exception as e:
    print(f"스택 출력 가져오기 오류: {e}")
    print("구성 변수를 수동으로 업데이트해 주세요")

# Strands 에이전트 생성
이 섹션에서는 Strands 프레임워크를 사용하여 에이전트를 생성합니다

#### 에이전트 구성 및 지침 정의

In [ ]:
protein_agent_name = 'Protein-Design-Agent-Strands'
protein_agent_description = "Strands 프레임워크를 사용한 AWS HealthOmics 워크플로우 기반 단백질 설계 및 최적화 에이전트"
protein_agent_instruction = """
당신은 방향성 진화 알고리즘을 사용하여 연구자들이 단백질 서열을 최적화하는 데 도움을 주는 단백질 설계 전문 AI입니다.
AWS HealthOmics 워크플로우를 트리거하여 단백질 서열 최적화를 수행하고 진행 상황을 모니터링할 수 있습니다.

당신의 기능은 다음과 같습니다:
1. 사용자 정의 매개변수로 단백질 설계 최적화 워크플로우 시작
2. 실행 중인 워크플로우의 상태 모니터링
3. 완료된 최적화에서 결과 검색 및 분석

단백질 서열 작업 시:
- 서열이 유효한 아미노산 문자만 포함하는지 검증
- 최적화 과정에 대한 명확한 설명 제공
- 사용자가 결과와 그 의미를 이해할 수 있도록 도움

항상 도움이 되고 단백질 설계 과정에 대한 자세한 정보를 제공하세요.
"""

#### Strands 에이전트용 도구 정의
이 도구들은 단백질 설계 작업을 위해 AWS HealthOmics 워크플로우를 호출합니다

In [ ]:
# Bedrock 에이전트 액션 그룹과 일치하는 업데이트된 도구 가져오기
from utils.protein_design_tools import trigger_aho_workflow, monitor_aho_workflow, test_configuration

print("도구가 성공적으로 가져와졌습니다:")
print(f"- trigger_aho_workflow: {trigger_aho_workflow.__doc__.split('Args:')[0].strip()}")
print(f"- monitor_aho_workflow: {monitor_aho_workflow.__doc__.split('Args:')[0].strip()}")

#### Strands 에이전트 생성

In [ ]:
# Bedrock 모델 생성
model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Bedrock 에이전트 액션 그룹과 일치하는 도구로 에이전트 생성
protein_agent = Agent(
    system_prompt=protein_agent_instruction,
    model=model,
    tools=[trigger_aho_workflow, monitor_aho_workflow]
)

print(f"Strands 프레임워크로 단백질 설계 에이전트를 생성했습니다")

# 에이전트 테스트
몇 가지 예제 쿼리로 단백질 설계 에이전트를 테스트해보겠습니다

In [ ]:
#https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/logs/
#import logging

# 루트 strands 로거 구성
#logging.getLogger("strands").setLevel(logging.DEBUG)

# 로그를 보기 위한 핸들러 추가
#logging.basicConfig(
#    format="%(levelname)s | %(name)s | %(message)s", 
#    handlers=[logging.StreamHandler()]
#)

#### 테스트 1: 단백질 최적화 시작

In [ ]:
# 단백질 최적화 시작 테스트
test_sequence = "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS"

query = f"이 단백질 서열을 최적화해 주세요: {test_sequence}"

print("쿼리:", query)

try:
    # 에이전트 실행
    print("\n응답:")
    protein_agent(query)
except Exception as e:
    print(f"에이전트 실행 중 오류: {e}")
    import traceback
    traceback.print_exc()

#### 테스트 2: 워크플로우 상태 모니터링

In [ ]:
# 워크플로우 모니터링 테스트 (이전 테스트의 실행 ID 사용)
# 'YOUR_RUN_ID'를 이전 테스트의 실제 실행 ID로 교체하세요
test_run_id = "{test_run_id}"  # 실제 실행 ID로 업데이트하세요

query = f"워크플로우 실행 {test_run_id}의 상태를 확인해 주세요"

print("쿼리:", query)
print("\n응답:")
response = protein_agent(query)

#### 테스트 3: 사용자 정의 매개변수를 사용한 고급 최적화

In [ ]:
# 사용자 정의 매개변수로 테스트
query = "ACDEFGHIKLMNPQRSTVWY 서열에 대해 20개의 병렬 체인과 200단계로 단백질 최적화를 실행해 주세요"

print("쿼리:", query)
print("\n응답:")
response = protein_agent(query)

In [ ]:
# 메트릭: https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/metrics/

# AgentResult를 통해 메트릭에 액세스
print(f"총 토큰: {response.metrics.accumulated_usage['totalTokens']}")
print(f"실행 시간: {sum(response.metrics.cycle_durations):.2f}초")
print(f"사용된 도구: {list(response.metrics.tool_metrics.keys())}")